In [ ]:
import stanza
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
# Columns that we want to use
columns = ['MET', 'PRA', 'ASE', 'EVA', 'RET', 'APR', 'DOM', 'REC', 'MEJ', 'GÉNERO ALUMNO', 'PROM ACUMULADO EN PROFESIONAL', 'Género del profesor', 'Por qué no lo recomendarias', 'Comentarios']
aspects = ['MET', 'PRA', 'ASE', 'EVA', 'RET', 'APR', 'DOM', 'REC', 'MEJ']

In [ ]:
# drive.mount('/content/drive')
# df=pd.read_excel('drive/MyDrive/Tesis/ecoa-demografico.xlsx')
df = pd.read_excel('ecoa-demografico.xlsx')

In [ ]:
df.head()

In [ ]:
print(f'Our original dataset is of shape: {df.shape}')

In [ ]:
df_ecoas = df[columns]
df_ecoas.head()

In [ ]:
pd.options.mode.chained_assignment = None # Disable warning
df_ecoas = df_ecoas.dropna(subset=['Por qué no lo recomendarias', 'Comentarios'], thresh=1) # Delete all rows where there are neither negative or positive comments
df_ecoas['GÉNERO ALUMNO'] = np.where((df_ecoas['GÉNERO ALUMNO'] == 'Femenino'), 0, df_ecoas['GÉNERO ALUMNO'])
df_ecoas['GÉNERO ALUMNO'] = np.where((df_ecoas['GÉNERO ALUMNO'] == 'Masculino'), 1, df_ecoas['GÉNERO ALUMNO'])
df_ecoas['Género del profesor'] = np.where((df_ecoas['Género del profesor'] == 'Femenino'), 0, df_ecoas['Género del profesor'])
df_ecoas['Género del profesor'] = np.where((df_ecoas['Género del profesor'] == 'Masculino'), 1, df_ecoas['Género del profesor'])

In [ ]:
for asp in aspects: # We delete all incomplete submissions
  df_ecoas.loc[df_ecoas[asp].isnull(),asp] = 99
  df_ecoas = df_ecoas.loc[df_ecoas[asp] != 99]
df_ecoas = df_ecoas.reset_index(drop=True) # Reset the index

In [ ]:
df_ecoas= df_ecoas.rename(columns={"Comentarios": "Positivos"})
df_ecoas= df_ecoas.rename(columns={"Por qué no lo recomendarias": "Negativos"})
p_only = df_ecoas['Positivos'].count()
n_only = df_ecoas['Negativos'].count()
print(f'Tenemos { p_only } comentarios SOLO positivos')
print(f'Tenemos { n_only } comentarios SOLO negativos')

In [ ]:
df_ecoas['Tipo Comentario'] = 0
df_ecoas.loc[pd.notna(df_ecoas['Positivos']), 'Tipo Comentario'] = 1
df_ecoas.loc[pd.notna(df_ecoas['Negativos']) & pd.notna(df_ecoas['Positivos']), 'Tipo Comentario'] = 2


In [ ]:
df_ecoas['Negativos'] = df_ecoas['Negativos'].fillna('')
df_ecoas['Positivos'] = df_ecoas['Positivos'].fillna('')
df_ecoas["Comentarios"] = df_ecoas["Positivos"] + ' ' + df_ecoas["Negativos"]
df_ecoas = df_ecoas.drop(columns=['Negativos', 'Positivos'])

In [ ]:
df_ecoas['AVG'] = df_ecoas[['APR', 'EVA', 'MET', 'PRA', 'REC', 'RET']].mean(axis=1).round(1)

In [ ]:
# This is the data set after initial cleaning
df_ecoas.head()

### Lemmatization
*In this step we performed some NLP preprocessing techquines such as stop words removal, symbols and special characters removal and lemmatization of the entire comments column*

In [ ]:
comments = df_ecoas['Comentarios']

for a in range(len(comments)):
  if type(comments[a]) is not str:
    print(type(comments[a]), 'No es str: ', a)
    df_ecoas.drop(index=a)

comments = comments.str.lower()

nltk.download('stopwords')
stop = stopwords.words('spanish')
stop.remove('no')
stop.append('es')

# Tokenize the comments
for i in range(len(comments)):
  comments[i] = str(comments[i]).split()
  comments[i] = [word for word in comments[i] if word not in stop]

# Remove the symbols and replace them with blank spaces
sim = "!\"#$%&()*+-.,/:;<=>?@[\]^_`{|}~\n"

# Para cada símbolo, se remplaza por blank space
for c in range(len(comments)):
  for i in range(len(comments[c])):
    for j in sim:
      if comments[c][i].__contains__(j):
        comments[c][i] = comments[c][i].replace(j,'')

comments

In [ ]:
total_less2  = 0
total_si = 0
total_no = 0

for comm in comments:
  if len(comm) <2:
    total_less2 += 1
    if 'si' in comm:
      total_si += 1
    if 'no' in comm:
      total_no += 1

print(f'Tenemos {total_less2} comentarios con una sola palabra, sin embargo, no los eliminaremos porque muchos son palabras que pueden ayudar al modelo')
print(f'Tenemos {total_si} comentarios con la palabra "si"')
print(f'Tenemos {total_no} comentarios con la palabra "no"')

In [ ]:
comments = list(comments)
for i in range(len(comments)):
  while '' in comments[i]:
    comments[i].remove('')

comentarios_merge=[None]*len(comments)

for i in range(len(comentarios_merge)):
  comentarios_merge[i]=(TreebankWordDetokenizer().detokenize(comments[i]))

print('Example of comment:')
comentarios_merge[2]

In [ ]:
comments_lemm = []

# Config the Stanza pipeline
config = {
	'processors': 'tokenize,mwt,pos,lemma',
  'lang': 'es',
}
nlp = stanza.Pipeline(**config) # Initialize the pipeline using a configuration dict

In [ ]:
import concurrent.futures

com_lemmatized = []
comments_lemm = []

def lemmatize_comments(comment):
  doc = nlp(comment)
  lemmatized_comment = []

  for sent in doc.sentences:
    lemmatized_comment.extend([word.lemma for word in sent.words])
  return lemmatized_comment

with concurrent.futures.ThreadPoolExecutor() as executor:
  results = executor.map(lemmatize_comments, comentarios_merge)

comments_lemm = list(results)


In [ ]:
print('Example of comment after lemmatizaion:')
comments_lemm[2]

In [ ]:
df_ecoas['Lemm'] = comments_lemm
df_ecoas.tail()

In [ ]:
df_ecoas.shape

In [ ]:
df_ecoas.tail()

In [ ]:
df_ecoas.to_csv('./datasets/df_ready.csv')